# KNN (K-nearest neighbours) algorithm

In [5]:
# import libraries
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import jax.numpy as jnp
import jax
from random import seed
from random import randrange

Three main steps:
1. calculate Euclidean distance;
2. get nearest neighbours;
3. make predictions.

## Define the distance

There are various kinds of distances, for instance Euclidean distance, Hamming distance, Manhattan distance and Minkowski distance. For the euclidean distance, I simply compute the L2 norm of two vectors (the rows of the dataset):

In [6]:
def euclidean_distance(row1, row2):
  ''' It computes the Euclidean distance between vectors row1 and row2.

  @param: row1 First vector.
  @param: row2 Second vector.  
  '''
  
  distance = 0.0
  for i in range(len(row1)-1): # go up to len(row)-1 because the last element of the row is the output
    distance += (row1[i] - row2[i])**2
  return sqrt(distance)

## Get nearest neighbours

The steps are:
1.   take a sample;
2.   compute the distance between such sample and every other sample in the dataset;
3. sort the samples (in descending order) according to the computed distances;
4. select the best k samples (the nearest neighbours).



In [7]:
def get_neighbors(train_set, sample, k):
  """ It returns a list containing the k nearest neighbours of sample in train.

  @param: train_set The training set from which extracting the neighbours.
  @param: sample The sample to consider; we want its neighbours.
  @param: k The number of nearest neighbours to extract.
  """

  distances = list() # here I store all the distances

  # compute the distances
  for train_row in train_set:
    dist = euclidean_distance(sample, train_row)
    distances.append((train_row, dist)) # in distances I append pairs of values row-distance

  # sort the samples:
  # key is a lambda which is executed prior to make comparisons; so we are sorting
  # the samples by the second (0-indexed) elements of objects in distances (namely, the distances)
  distances.sort(key=lambda tup: tup[1]) 

  # get the neighbours
  neighbors = list()
  for i in range(k):
    neighbors.append(distances[i][0]) # append only the sample (not the distance)

  return neighbors

## Make predictions

It is a classification task: we just have to compute the most frequent class in the neighborood to carry out the prediction:

In [8]:
def predict_classification(training_set, sample, k):
  '''It provides a prediction for sample using the training_set and the best k neighbours.

  @param: training_set It is the training set to use for providing a prediction for sample.
  @param: sample The sample for which providing the prediction.
  @param: k The hyperparameter of the algorithm; it represents the number of nearest neighbours to use.
  '''

  # get the k-nearest neighbours
  neighbors = get_neighbors(training_set, sample, k)

  # get the classes of the k-nearest neighbours
  output_values = [row[-1] for row in neighbors] # I take row[-1] because the class is the last entry of the row

  # perform the prediction: I take the most frequent class (output value)
  prediction = max(set(output_values), key=output_values.count)
  
  return prediction

## Algorithm

In [9]:
# KNN Algorithm
def k_nearest_neighbors(training_set, test_set, k):
	"""
	It applies the KNN algorithm to each entry of the test_set using the training_set as training set
	and using k as hyperparameter.

	@param: training_set It is the training set to use.
	@param: test_set It is the test set; this function provides a prediction for each entry of the test_set.
	@param: k It represents the main hyperparameter of the algorithm, namely it says how many nearest neighbours
	to use for computing the prediction.
	"""

	# we return a list of predictions
	predictions = list()
 
	# compute a prediction for each entry of the test_set
	for row in test_set:
		output = predict_classification(training_set, row, k)
		predictions.append(output)
	
	return(predictions)

# Preprocess data - Random under sampling

The two main approaches to randomly resampling an imbalanced dataset are to delete examples from the majority class, called undersampling, and to duplicate examples from the minority class, called oversampling.


*   Random oversampling duplicates examples from the minority class in the training dataset and can result in overfitting for some models.
*   Random undersampling deletes examples from the majority class and can result in losing information invaluable to a model.

Random oversampling involves randomly selecting examples from the minority class, with replacement, and adding them to the training dataset. Random undersampling involves randomly selecting examples from the majority class and deleting them from the training dataset.



In [10]:
def RUS(ratio, set):
  '''It returns a list obtained by merging two lists, one containing the fraud set and the
  other containing the non-fraud set.
  The ratio parameter defines the ratio fraud/non-fraud.
  e.g.: ratio=1 and len(fraud)= 100 implies that len(new_not_fraud)=100.
  
  @param: ratio It is the ratio between fraud and non-fraud data; it is defined as: ratio:=fraud_set/non-fraud_set.
  @param set It is the set to split.
  '''

  # initialize the two lists
  fraud_set = list()
  non_fraud_set = list()

  # for each sample in the set, append it to the proper class
  for sample in set:
    if sample[30] == 1:
      fraud_set.append(sample)
    else:
      non_fraud_set.append(sample)

  # compute new length of non_fraud_set
  new_length_non_fraud_set = int(len(fraud_set) / ratio)
  print('new_length: ', new_length_non_fraud_set)

  # shuffle non_fraud_set
  np.random.seed(0) # set seed to 0 for reproducibility
  np.random.shuffle(non_fraud_set)

  # take the first new_length_non_fraud_set elements from non_fraud_set
  non_fraud_reduced_set = non_fraud_set[0:new_length_non_fraud_set]

  # create a unique list for the new set
  new_set = fraud_set
  new_set.extend(non_fraud_reduced_set)

  # return the whole new set
  return new_set

# Load the dataset and split it into training and test set

First step is the definition of the hyperparameters.

In [11]:
# test_set / training_set
fraction_validation = 0.3 # 30% of test set and 70% of training set

# inside the training_set: fraud / non-fraud
ratio1 = 1 # 50:50
ratio2 = 34/66 # 34:66
ratio3 = 1/3 # 25:75

# hyperparameter k
k = 10

Load the dataset and divide it:

In [12]:
# load dataset
dataset = pd.read_csv('creditcard.csv')

# normalize it
# TODO

# parse to numpy and shuffle it
dataset = dataset.to_numpy() # not normalized
np.random.seed(10)
np.random.shuffle(dataset) # shuffle the rows

# divide into training_set and test_set
num_train = int(dataset.shape[0] * (1 - fraction_validation))
training_set = dataset[:num_train,:] # take the first num_train rows
test_set = dataset[num_train:,:] # take the last tot-num_train rows

# apply random under sampling to the training set
training_set1 = RUS(ratio1, training_set)
training_set2 = RUS(ratio2, training_set)
training_set3 = RUS(ratio3, training_set)

# apply random under sampling to the test set
test_set1 = RUS(ratio1, test_set)
test_set2 = RUS(ratio2, test_set)
test_set3 = RUS(ratio3, test_set)

new_length:  173
new_length:  335
new_length:  519
new_length:  71
new_length:  137
new_length:  213


# Apply the algorithm

In [185]:
# proportion 1: 50:50
predictions1 = k_nearest_neighbors(training_set1, test_set1, k)

In [186]:
# proportion 2: 34:66
predictions2 = k_nearest_neighbors(training_set2, test_set2, k)

In [188]:
# proportion 3: 25:75
predictions3 = k_nearest_neighbors(training_set3, test_set3, k)

# Evaluate algorithm through cross-validation

**Cross validation:**

Cross-validation, sometimes called rotation estimation or out-of-sample testing, is any of various similar model validation techniques for assessing how the results of a statistical analysis will generalize to an independent data set. Cross-validation is a resampling method that uses different portions of the data to test and train a model on different iterations. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice.

**k-fold cross-validation:**

In k-fold cross-validation, the original sample is randomly partitioned into k equal sized subsamples. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k − 1 subsamples are used as training data. The cross-validation process is then repeated k times, with each of the k subsamples used exactly once as the validation data. The k results can then be averaged to produce a single estimation. k = 10 is usually used.

Create functions for evaluating the algorithm:

In [21]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
  '''It returns the n_folds folds.
  
  @param: dataset It is the dataset to split.
  @param: n_folds The number of folds to create.
  '''

  # the returned value will be a list of lists (folds)
  dataset_split = list()

  # create a list from the dataset
  dataset_copy = dataset.tolist()

  # compute the number of elements in each fold
  fold_size = int(len(dataset) / n_folds)

  # build each fold by popping values from the copy of the dataset
  for _ in range(n_folds):
  	fold = list()
    # build the fold
  	while len(fold) < fold_size:
  		index = randrange(len(dataset_copy))
  		fold.append(dataset_copy.pop(index))
    # append the fold
  	dataset_split.append(fold)
   
  return dataset_split

In [14]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	"""
	It computes the accuracy between the two provided lists.
	Accuracy := n_correct / n_actual

	@param: actual It is the list with all the actual values.
	@param: predicted It is the list with all the predicted values.
	"""

	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	
	return correct / float(len(actual)) * 100.0

In [15]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
  '''It returns a list of accuracies, each of them computed on a certain fold.
  
  @param: dataset It is the dataset on which evaluating the algorithm.
  @param: algorithm It is the algorithm to evaluate; it is a lambda.
  @param: n_folds It represents the number of folds on which divide the dataset.
  @param: args It contains parameters of the algorithm (k in the case of KNN).
  '''

  # split the dataset into n_folds
  folds = cross_validation_split(dataset, n_folds)

  # this list contains the accuracy of the algorithm computed for each fold at a time
  scores = list()

  # for each fold compute the accuracy
  for fold in folds:
    training_set = list(folds) # copy the dataset
    training_set.remove(fold) # remove the current fold
    training_set = sum(training_set, []) # merge all the folds in training_set in one list
    test_set = list()

    # build the test_set by removing the class in each row of the current fold
    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      row_copy[-1] = None

    # compute the accuracy and append it to scores
    predicted = algorithm(training_set, test_set, *args)
    actual = [row[-1] for row in fold] # actual values are in the last column of the fold

    # compute the accuracy
    accuracy = accuracy_metric(actual, predicted)    
    scores.append(accuracy)                 

  return scores

Evaluate the algorithm:

In [22]:
n_folds = 3
rus_dataset = RUS(0.5,dataset)
rus_dataset = np.asarray(rus_dataset)

# I pass the result of the RUS function on the dataset
scores = evaluate_algorithm(rus_dataset, k_nearest_neighbors, n_folds, k)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

new_length:  488
Scores: [69.67213114754098, 75.81967213114754, 73.36065573770492]
Mean Accuracy: 72.951%
